In [1]:
import os
import osmnx as ox
import geopandas as gpd
import random
from folium.plugins import MarkerCluster
import csv
import numpy as np
import geopandas as gpd
import networkx as nx
import pandas as pd
from shapely.geometry import Point, LineString, mapping
from tqdm import tqdm
from scipy.spatial import cKDTree
from pyproj import Transformer
import folium

In [ ]:
# --- Define location ---
place_name = "Kolkata, West Bengal, India"

# --- Download road network ---
G = ox.graph_from_place(place_name, network_type='drive')
roads = ox.graph_to_gdfs(G, nodes=False, edges=True)

# --- Project roads for accurate centroid calculation ---
roads_proj = roads.to_crs(roads.estimate_utm_crs())
city_center = roads_proj.geometry.centroid.to_crs(epsg=4326)
center = [city_center.y.mean(), city_center.x.mean()]

# --- Create Folium map centered on the city ---
m = folium.Map(location=center, zoom_start=12)

# --- Download hospitals (POIs) ---
tags = {'amenity': 'hospital'}
hospitals = ox.features.features_from_place(place_name, tags)

# --- Filter only point geometries for hospitals ---
hospitals_points = hospitals[hospitals.geometry.geom_type == 'Point']

# --- Add road network to map ---
for _, row in roads.iterrows():
    if row.geometry.geom_type == 'LineString':
        coords = [(y, x) for x, y in row.geometry.coords]
        folium.PolyLine(coords, color="blue", weight=2, opacity=0.6).add_to(m)

# --- Add hospitals to map ---
for _, row in hospitals_points.iterrows():
    folium.Marker(
        [row.geometry.y, row.geometry.x],
        popup=row['name'] if 'name' in row else 'Hospital',
        icon=folium.Icon(color='red', icon='plus')
    ).add_to(m)

# --- Prepare output directory ---
output_dir = r"E:\NetworkAnalysis\"
os.makedirs(output_dir, exist_ok=True)

# --- Save roads and hospitals with all attributes using GeoPackage ---
roads.to_file(os.path.join(output_dir, "kolkata_roads.gpkg"), layer='roads', driver="GPKG")
hospitals_points.to_file(os.path.join(output_dir, "kolkata_hospitals.gpkg"), layer='hospitals', driver="GPKG")

# --- Display map ---
m

In [ ]:
# --- Parameters ---
num_points = 1000
min_distance = 10  # meters
output_file = os.path.join(output_dir, "accident_points.gpkg")

# --- Ensure roads are projected for distance calculations ---
roads_proj = roads.to_crs(roads.estimate_utm_crs())

# --- Create list of all line segments along roads ---
lines = roads_proj.geometry.values
all_lengths = [line.length for line in lines]

# --- Generate random points along roads ---
points_list = []
while len(points_list) < num_points:
    # Randomly pick a road segment proportional to length
    idx = random.choices(range(len(lines)), weights=all_lengths, k=1)[0]
    line = lines[idx]

    # Pick a random distance along the line
    random_dist = random.uniform(0, line.length)
    point = line.interpolate(random_dist)

    # Check minimum distance constraint
    if all(point.distance(p) >= min_distance for p in points_list):
        points_list.append(point)

# --- Create GeoDataFrame ---
points_gdf = gpd.GeoDataFrame(geometry=points_list, crs=roads_proj.crs)

# --- Reproject to EPSG:32645 ---
points_gdf = points_gdf.to_crs(epsg=32645)

# --- Save as GeoPackage ---
points_gdf.to_file(output_file, layer="accident_points", driver="GPKG")

print(f"Saved {len(points_gdf)} random points to {output_file} (EPSG:32645)")

In [ ]:
# --- Map center based on roads centroid ---
roads_proj = roads.to_crs(roads.estimate_utm_crs())
city_center = roads_proj.geometry.centroid.to_crs(epsg=4326)
center = [city_center.y.mean(), city_center.x.mean()]

# --- Create Folium map ---
m = folium.Map(location=center, zoom_start=12, tiles="CartoDB positron")

# --- Add roads as a separate layer ---
roads_fg = folium.FeatureGroup(name="Roads")
for _, row in roads.iterrows():
    if row.geometry.geom_type == 'LineString':
        coords = [(y, x) for x, y in row.geometry.coords]
        folium.PolyLine(coords, color='blue', weight=2, opacity=0.6).add_to(roads_fg)
m.add_child(roads_fg)

# --- Add hospitals as MarkerCluster ---
hospitals_fg = folium.FeatureGroup(name="Hospitals")
hospital_cluster = MarkerCluster().add_to(hospitals_fg)
for _, row in hospitals_points.iterrows():
    folium.Marker(
        [row.geometry.y, row.geometry.x],
        popup=row['name'] if 'name' in row else 'Hospital',
        icon=folium.Icon(color='red', icon='plus')
    ).add_to(hospital_cluster)
m.add_child(hospitals_fg)

# --- Add accidents as MarkerCluster or CircleCluster ---
accidents_fg = folium.FeatureGroup(name="Accident Points")
accident_cluster = MarkerCluster().add_to(accidents_fg)
for _, row in points_gdf.to_crs(epsg=4326).iterrows():
    folium.CircleMarker(
        [row.geometry.y, row.geometry.x],
        radius=4,
        color='orange',
        fill=True,
        fill_opacity=0.7,
        popup="Accident"
    ).add_to(accident_cluster)
m.add_child(accidents_fg)

# --- Add layer control for interactivity ---
folium.LayerControl(collapsed=False).add_to(m)

# --- Display map ---
m

In [6]:
# ======================
# 1. Load Data
# ======================
Input_dir = r"E:\NetworkAnalysis"
Output_dir = r"E:\NetworkAnalysis\Results"
os.makedirs(Output_dir, exist_ok=True)

# Load road network
roads = gpd.read_file(os.path.join(Input_dir, "kolkata_roads.gpkg"), layer="roads")

# Load hospitals and add unique FID
hospitals = gpd.read_file(os.path.join(Input_dir, "kolkata_hospitals.gpkg"))
hospitals = hospitals.reset_index().rename(columns={"index": "fid"})

# Load accident points and add unique FID
accidents = gpd.read_file(os.path.join(Input_dir, "accident_points.gpkg"))
accidents = accidents.reset_index().rename(columns={"index": "fid"})

print("Data loaded successfully!")
print("Roads:", len(roads))
print("Hospitals:", len(hospitals))
print("Accidents:", len(accidents))

Data loaded successfully!
Roads: 90950
Hospitals: 147
Accidents: 1000


In [3]:
# ------------------------
# Config - change these
# ------------------------
roads_fp = r"E:\NetworkAnalysis\kolkata_roads.gpkg"
orig_fp  = r"E:\NetworkAnalysis\accident_points.gpkg"
dest_fp  = r"E:\NetworkAnalysis\kolkata_hospitals.gpkg"
out_csv  = r"E:\NetworkAnalysis\od_matrix.csv"
save_example_route_html = "od_route_example.html"
n_sample_routes = 5

In [5]:
# ------------------------
# 1. Load layers & sanity checks
# ------------------------
roads = gpd.read_file(roads_fp)
origins = gpd.read_file(orig_fp)
destinations = gpd.read_file(dest_fp)

print(f"Roads: {len(roads)} features; Origins: {len(origins)}; Destinations: {len(destinations)}")

# Ensure geometries are LineString / Point: explode Multi* into single parts
roads = roads.explode(index_parts=False).reset_index(drop=True)
origins = origins.explode(index_parts=False).reset_index(drop=True)
destinations = destinations.explode(index_parts=False).reset_index(drop=True)

Roads: 90950 features; Origins: 1000; Destinations: 147


In [6]:
# ------------------------
# 2. Ensure metric CRS (meters)
# ------------------------
# If CRS is None, stop
if roads.crs is None:
    raise ValueError("roads.shp has no CRS. Please set the correct CRS before running.")
    
metric_crs = "EPSG:32645"
roads = roads.to_crs(metric_crs)
origins = origins.to_crs(metric_crs)
destinations = destinations.to_crs(metric_crs)

In [8]:
# ------------------------
# 3. Build node index and edges (efficient unique node ids)
# ------------------------
nodes = {}             
node_id_counter = 0
edges = []             

for geom in roads.geometry:
    # skip invalid or empty
    if geom is None or geom.is_empty:
        continue
    coords = list(geom.coords)
    for i in range(len(coords) - 1):
        a = coords[i]
        b = coords[i + 1]
        if a not in nodes:
            nodes[a] = node_id_counter
            node_id_counter += 1
        if b not in nodes:
            nodes[b] = node_id_counter
            node_id_counter += 1
        u = nodes[a]
        v = nodes[b]
        seg_len = LineString([a, b]).length
        edges.append((u, v, seg_len, (a, b)))

print(f"Built raw node list: {len(nodes)} nodes, edges: {len(edges)}")


Built raw node list: 125726 nodes, edges: 263510


In [9]:
# ------------------------
# 4. Create networkx Graph
# ------------------------
G = nx.Graph()
# add nodes with position attribute
for coord, nid in nodes.items():
    G.add_node(nid, x=coord[0], y=coord[1])

# add edges
for u, v, length, (a, b) in edges:
    # if multiple edges between same nodes, keep smallest weight
    if G.has_edge(u, v):
        if length < G[u][v]['weight']:
            G[u][v]['weight'] = length
            G[u][v]['geometry'] = LineString([a, b])
    else:
        G.add_edge(u, v, weight=length, geometry=LineString([a, b]))

print(f"Graph built successfully!")
print(f"Nodes: {G.number_of_nodes()}, Edges: {G.number_of_edges()}")

Graph built successfully!
Nodes: 125726, Edges: 137185


In [10]:
# ------------------------
# 5. Build KD-tree for node snapping
# ------------------------
coord_list = list(nodes.keys())               
node_id_list = [nodes[c] for c in coord_list] 
node_coords = np.array(coord_list)           
kdtree = cKDTree(node_coords)

def snap_point_to_node(point):
    """
    point: shapely Point in metric_crs
    returns: node_id
    """
    dist, idx = kdtree.query([point.x, point.y])
    return node_id_list[idx]

# Precompute origin/destination node ids (vectorized)
orig_coords = np.column_stack([origins.geometry.x, origins.geometry.y])
dest_coords = np.column_stack([destinations.geometry.x, destinations.geometry.y])
_, orig_idx = kdtree.query(orig_coords)
_, dest_idx = kdtree.query(dest_coords)
orig_node_ids = [node_id_list[i] for i in orig_idx]
dest_node_ids = [node_id_list[i] for i in dest_idx]

In [42]:
# ------------------------
# 6. Compute OD distances (efficiently, constant speed for all roads)
# ------------------------
import os

# Set constant travel speed
speed_kmh = 40  # constant speed (same for all roads)
speed_mps = speed_kmh * 1000 / 3600  # convert km/h to m/s

# Prepare CSV writer
write_header = not os.path.exists(out_csv)
with open(out_csv, "w", newline="") as fout:
    writer = csv.writer(fout)
    writer.writerow([
        "origin_id", "destination_id", "distance_m", "speed_kmh", "time_min", "zone"
    ])
    
    # Iterate over each origin
    for oi, o_node in enumerate(tqdm(orig_node_ids, desc="Processing origins")):
        # Dijkstra gives shortest path distance (in meters)
        lengths = nx.single_source_dijkstra_path_length(G, o_node, weight="weight")
        
        # For each destination
        for di, d_node in enumerate(dest_node_ids):
            dist = lengths.get(d_node, None)  # None if unreachable
            
            if dist is not None:
                # Compute travel time (minutes) using constant speed
                time_min = (dist / speed_mps) / 60
                
                # Zone classification
                if time_min <= 20:
                    zone = "Safe Zone"
                elif time_min <= 30:
                    zone = "Warning Zone"
                else:
                    zone = "Danger Zone"
            else:
                time_min = None
                zone = "Unreachable"
            
            writer.writerow([
                oi,        
                di,        
                dist,
                speed_kmh, 
                time_min,
                zone
            ])

print(f"Saved OD CSV to: {out_csv}")

Processing origins: 100%|██████████████████████████████████████████████████████████| 1000/1000 [23:55<00:00,  1.44s/it]

Saved OD CSV to: E:\NetworkAnalysis\od_matrix.csv


In [11]:
import os
import geopandas as gpd
from shapely.geometry import LineString, Point
from tqdm import tqdm
import networkx as nx

# ------------------------
# Config
# ------------------------
out_gpkg = r"E:\NetworkAnalysis\od_routes_.gpkg"
speed_kmh = 40  # constant speed for all roads
speed_mps = speed_kmh * 1000 / 3600  # convert km/h to m/s

# CRS (adjust to match your data)
crs = "EPSG:32645"

# ------------------------
# Initialize GeoDataFrames for each zone
# ------------------------
layers = {
    "Safe_Routes": gpd.GeoDataFrame(columns=["origin_id","destination_id","distance_m","time_min","geometry"], crs=crs),
    "Safe_Origins": gpd.GeoDataFrame(columns=["origin_id","geometry"], crs=crs),
    "Safe_Destinations": gpd.GeoDataFrame(columns=["destination_id","geometry"], crs=crs),
    
    "Warning_Routes": gpd.GeoDataFrame(columns=["origin_id","destination_id","distance_m","time_min","geometry"], crs=crs),
    "Warning_Origins": gpd.GeoDataFrame(columns=["origin_id","geometry"], crs=crs),
    "Warning_Destinations": gpd.GeoDataFrame(columns=["destination_id","geometry"], crs=crs),
    
    "Danger_Routes": gpd.GeoDataFrame(columns=["origin_id","destination_id","distance_m","time_min","geometry"], crs=crs),
    "Danger_Origins": gpd.GeoDataFrame(columns=["origin_id","geometry"], crs=crs),
    "Danger_Destinations": gpd.GeoDataFrame(columns=["destination_id","geometry"], crs=crs),
}

# ------------------------
# Compute OD distances & zones
# ------------------------
for oi, o_node in enumerate(tqdm(orig_node_ids, desc="Computing OD zones")):
    # Single-source shortest path lengths & paths
    lengths, paths = nx.single_source_dijkstra(G, o_node, weight="weight")
    
    for di, d_node in enumerate(dest_node_ids):
        dist = lengths.get(d_node, None)
        if dist is None:
            continue  # skip unreachable

        # Travel time in minutes
        time_min = (dist / speed_mps) / 60

        # Zone classification
        if time_min <= 20:
            zone = "Safe"
        elif time_min <= 30:
            zone = "Warning"
        else:
            zone = "Danger"

        # Path coordinates
        path_nodes = paths[d_node]
        path_coords = [(G.nodes[n]["x"], G.nodes[n]["y"]) for n in path_nodes]

        # Skip invalid paths (single-node)
        if len(path_coords) <= 1:
            continue

        route_geom = LineString(path_coords)

        # Origin & Destination points
        o_x, o_y = G.nodes[o_node]["x"], G.nodes[o_node]["y"]
        d_x, d_y = G.nodes[d_node]["x"], G.nodes[d_node]["y"]
        origin_geom = Point(o_x, o_y)
        dest_geom = Point(d_x, d_y)

        # Append to correct GeoDataFrames
        layers[f"{zone}_Routes"].loc[len(layers[f"{zone}_Routes"])] = [
            oi, di, dist, time_min, route_geom
        ]
        layers[f"{zone}_Origins"].loc[len(layers[f"{zone}_Origins"])] = [
            oi, origin_geom
        ]
        layers[f"{zone}_Destinations"].loc[len(layers[f"{zone}_Destinations"])] = [
            di, dest_geom
        ]

# ------------------------
# Save all layers to GPKG (with CRS enforced)
# ------------------------
for lname, gdf in layers.items():
    if not gdf.empty:
        gdf = gdf.set_crs(crs, allow_override=True)  # ensure CRS is attached
        gdf.to_file(out_gpkg, layer=lname, driver="GPKG")

print(f"✅ Saved OD routes + origins + destinations by zone into: {out_gpkg}")

Computing OD zones: 100%|████████████████████████████████████████████████████████| 1000/1000 [3:21:36<00:00, 12.10s/it]


✅ Saved OD routes + origins + destinations by zone into: E:\NetworkAnalysis\od_routes_.gpkg


In [17]:
import folium
import geopandas as gpd

# Load saved layers
gpkg_path = r"E:\NetworkAnalysis\od_routes_.gpkg"
layers = {
    "Safe_Routes": gpd.read_file(gpkg_path, layer="Safe_Routes"),
    "Warning_Routes": gpd.read_file(gpkg_path, layer="Warning_Routes"),
    "Danger_Routes": gpd.read_file(gpkg_path, layer="Danger_Routes"),
}

# Initialize map centered on first route
m = folium.Map(location=[layers["Safe_Routes"].geometry.iloc[0].centroid.y,
                         layers["Safe_Routes"].geometry.iloc[0].centroid.x],
               zoom_start=11)

# Define colors
zone_colors = {"Safe": "green", "Warning": "orange", "Danger": "red"}

# Add each layer with different color
for zone, gdf in layers.items():
    color = zone_colors[zone.split("_")[0]]
    for _, row in gdf.iterrows():
        folium.GeoJson(
            row.geometry,
            style_function=lambda x, c=color: {"color": c, "weight": 3},
            tooltip=f"🛣 Origin {row.origin_id} → Dest {row.destination_id}<br>"
                    f"📏 {row.distance_m:.0f} m<br>"
                    f"⏱ {row.time_min:.1f} min"
        ).add_to(m)

# Save map
m.save("od_routes_map.html")
print("✅ Interactive map saved as od_routes_map.html")


✅ Interactive map saved as od_routes_map.html
